In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import warnings
warnings.simplefilter("ignore", gpytorch.utils.warnings.NumericalWarning)

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Training data is 11 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 1000)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2

In [3]:
HAVE_KEOPS = False

In [4]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()

        if HAVE_KEOPS:
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.keops.RBFKernel())
        else:
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [5]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 50

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

/Users/jeewookim/.local/lib/python3.7/site-packages/gpytorch/utils/linear_cg.py:278: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [1, 11].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  curr_conjugate_vec,


Iter 1/50 - Loss: 0.862   lengthscale: 0.693   noise: 0.693
Iter 2/50 - Loss: 0.817   lengthscale: 0.644   noise: 0.644
Iter 3/50 - Loss: 0.770   lengthscale: 0.598   noise: 0.598
Iter 4/50 - Loss: 0.716   lengthscale: 0.554   noise: 0.554
Iter 5/50 - Loss: 0.668   lengthscale: 0.513   noise: 0.513
Iter 6/50 - Loss: 0.617   lengthscale: 0.474   noise: 0.474
Iter 7/50 - Loss: 0.576   lengthscale: 0.439   noise: 0.437
Iter 8/50 - Loss: 0.534   lengthscale: 0.408   noise: 0.402
Iter 9/50 - Loss: 0.491   lengthscale: 0.380   noise: 0.370
Iter 10/50 - Loss: 0.453   lengthscale: 0.356   noise: 0.339
Iter 11/50 - Loss: 0.414   lengthscale: 0.335   noise: 0.311
Iter 12/50 - Loss: 0.376   lengthscale: 0.317   noise: 0.285
Iter 13/50 - Loss: 0.339   lengthscale: 0.301   noise: 0.261
Iter 14/50 - Loss: 0.298   lengthscale: 0.288   noise: 0.238
Iter 15/50 - Loss: 0.263   lengthscale: 0.276   noise: 0.217
Iter 16/50 - Loss: 0.225   lengthscale: 0.266   noise: 0.198
Iter 17/50 - Loss: 0.184   length

In [6]:
test_n = 10000

test_x = torch.linspace(0, 1, test_n)
if torch.cuda.is_available():
    test_x = test_x.cuda()
print(test_x.shape)

torch.Size([10000])


In [7]:
import time

model.train()
likelihood.train()

# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood

test_x.requires_grad_(True)

with torch.no_grad():
    observed_pred = likelihood(model(test_x))

    # All relevant settings for using CIQ.
    #   ciq_samples(True) - Use CIQ for sampling
    #   num_contour_quadrature(10) -- Use 10 quadrature sites (Q in the paper)
    #   minres_tolerance -- error tolerance from minres (here, <0.01%).
    print("Running with CIQ")
    with gpytorch.settings.ciq_samples(True), gpytorch.settings.num_contour_quadrature(10), gpytorch.settings.minres_tolerance(1e-4):
        %time y_samples = observed_pred.rsample()

Running with CIQ


/Users/jeewookim/.local/lib/python3.7/site-packages/gpytorch/utils/linear_cg.py:278: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [1, 1].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:23.)
  curr_conjugate_vec,
/Users/jeewookim/.local/lib/python3.7/site-packages/gpytorch/utils/contour_integral_quad.py:86: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else '

CPU times: user 2.59 s, sys: 787 ms, total: 3.38 s
Wall time: 2.63 s


In [ ]:
import matplotlib.pyplot as plt

test_x = torch.linspace(0, 1, self.sample_rate)

        f, ax = plt.subplots(1, 1, figsize=(12, 6))
        ax.plot(self.train_x, self.train_y, 'k*')
        ax.set_ylim([-5, 7.5])

        if is_sample:
            ax.plot(test_x.numpy(), data)
            ax.legend(['Data', 'Sample from posterior'])
        else:
            mean, (lower, upper) = data
            ax.plot(test_x.numpy(), mean, 'b')
            ax.fill_between(test_x.numpy(), lower, upper, alpha=0.5)
            ax.legend(['Observed Data', 'Mean', 'Confidence'])